# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 64 new papers today
          19 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/18 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.10207


extracting tarball to tmp_2302.10207...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.10217


extracting tarball to tmp_2302.10217...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.10838


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.10714/sn-article.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2302.10838...

 done.


/tmp/ipykernel_2047/4030337529.py:34: LatexWarning: 2302.10838 did not run properly
sequence item 1: expected str instance, TexNode found
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10528-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10528) | **The CARMENES search for exoplanets around M dwarfs. Guaranteed time  observations Data Release 1 (2016-2020)**  |
|| I. Ribas, et al. -- incl., <mark>J. I. González Hernández</mark>, <mark>Th. Henning</mark>, <mark>E. Herrero</mark> |
|*Appeared on*| *2023-02-22*|
|*Comments*| *Published in A&A (this https URL), 25 pages, 12 figures, Tables 1 and 2 only available online*|
|**Abstract**| The CARMENES instrument was conceived to deliver high-accuracy radial velocity (RV) measurements with long-term stability to search for temperate rocky planets around a sample of nearby cool stars. The broad wavelength coverage was designed to provide a range of stellar activity indicators to assess the nature of potential RV signals and to provide valuable spectral information to help characterise the stellar targets. The CARMENES Data Release 1 (DR1) makes public all observations obtained during the CARMENES guaranteed time observations, which ran from 2016 to 2020 and collected 19,633 spectra for a sample of 362 targets. The CARMENES survey target selection was aimed at minimising biases, and about 70% of all known M dwarfs within 10 pc and accessible from Calar Alto were included. The data were pipeline-processed, and high-level data products, including 18,642 precise RVs for 345 targets, were derived. Time series data of spectroscopic activity indicators were also obtained. We discuss the characteristics of the CARMENES data, the statistical properties of the stellar sample, and the spectroscopic measurements. We show examples of the use of CARMENES data and provide a contextual view of the exoplanet population revealed by the survey, including 33 new planets, 17 re-analysed planets, and 26 confirmed planets from transiting candidate follow-up. A subsample of 238 targets was used to derive updated planet occurrence rates, yielding an overall average of 1.44+/-0.20 planets with 1 M_Earth < M sin i < 1000 M_Earth and 1 d < P_orb < 1000 d per star, and indicating that nearly every M dwarf hosts at least one planet. CARMENES data have proven very useful for identifying and measuring planetary companions as well as for additional applications, such as the determination of stellar properties, the characterisation of stellar activity, and the study of exoplanet atmospheres. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10207-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10207) | **Two New White Dwarfs With Variable Magnetic Balmer Emission Lines**  |
|| Joshua S. Reding, et al. -- incl., <mark>J. J. Hermes</mark>, <mark>R. J. Hegedus</mark> |
|*Appeared on*| *2023-02-22*|
|*Comments*| *7 pages, 3 figures. Submitted to MNRAS Jan. 5, referee report recommending publication received Feb. 1, resubmitted Feb. 20*|
|**Abstract**| We report the discovery of two apparently isolated stellar remnants that exhibit rotationally modulated magnetic Balmer emission, adding to the emerging DAHe class of white dwarf stars. While the previously discovered members of this class show Zeeman-split triplet emission features corresponding to single magnetic field strengths, these two new objects exhibit significant fluctuations in their apparent magnetic field strengths with variability phase. The Zeeman-split hydrogen emission lines in LP $705{-}64$ broaden from $9.4$ MG to $22.2$ MG over an apparent spin period of $72.629$ minutes. Similarly, WD J$143019.29{-}562358.33$ varies from $5.8$ MG to $8.9$ MG over its apparent $86.394$-minute rotation period. This brings the DAHe class of white dwarfs to at least five objects, all with effective temperatures within $500$ K of $8000$ K and masses ranging from $0.65{-}0.83\,M_{\odot}$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10217-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10217) | **The JWST Advanced Deep Extragalactic Survey: Discovery of an Extreme  Galaxy Overdensity at $z = 5.4$ with JWST/NIRCam in GOODS-S**  |
|| <mark>Jakob M. Helton</mark>, et al. |
|*Appeared on*| *2023-02-22*|
|*Comments*| *Submitted to ApJ; main text has 14 pages, 6 figures and 1 table; appendix has 1 page, 2 figures, and 2 tables*|
|**Abstract**| We report the discovery of an extreme galaxy overdensity at $z = 5.4$ in the GOODS-S field using JWST/NIRCam imaging from JADES and JEMS alongside JWST/NIRCam wide field slitless spectroscopy from FRESCO. We identified potential members of the overdensity using HST+JWST photometry spanning $\lambda = 0.4-5.0\ \mu\mathrm{m}$. These data provide accurate and well-constrained photometric redshifts down to $m \approx 29-30\,\mathrm{mag}$. We subsequently confirmed $N = 96$ potential members of the overdensity using JWST slitless spectroscopy over $\lambda = 3.9-5.0\ \mu\mathrm{m}$ through a targeted line search for $\mathrm{H} \alpha$ around the best-fit photometric redshift. We verified that $N = 53$ galaxies reside in the field at $z = 5.2-5.5$ while $N = 43$ galaxies reside in an overdensity at $z = 5.4$ around $\sim 10-12$ times that of a random volume. Stellar populations for these galaxies were inferred from the photometry and used to construct the star-forming main sequence, where protocluster members appeared more massive and exhibited earlier star formation (and thus older stellar populations) when compared to their field galaxy counterparts. We estimate the total halo mass of this large-scale structure to be $13.0 \lesssim \mathrm{log}_{10} \left( M_{\mathrm{halo}}/M_{\odot} \right) \lesssim 13.5$ using an empirical stellar mass to halo mass relation, although this is likely an underestimate as a result of incompleteness. Our discovery demonstrates the power of JWST at constraining dark matter halo assembly and galaxy formation at very early cosmic times. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10664-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10664) | **TOI-1055 b: a neptunian planet characterised with HARPS, TESS, and  CHEOPS**  |
|| A. Bonfanti, et al. -- incl., <mark>M. B. Davies</mark>, <mark>M. Güdel</mark> |
|*Appeared on*| *2023-02-22*|
|*Comments*| *13 pages, 6 figures, 5 tables. Accepted for publication in A&A*|
|**Abstract**| TOI-1055 is a Sun-like star known to host a transiting Neptune-sized planet on a 17.5-day orbit (TOI-1055 b). Radial velocity (RV) analyses carried out by two independent groups using almost the same set of HARPS spectra provide planetary masses that differ by $\sim$2$\sigma$. Our aim was to solve the inconsistency in the published planetary masses by significantly extending the set of HARPS RV measurements and employing a new analysis tool able to account and correct for stellar activity. Our further aim was to improve the precision on the planetary radius by observing two transits of the planet with the CHEOPS space telescope. We fitted a skew normal function to each cross correlation function extracted from the HARPS spectra to obtain RV measurements and hyperparameters to be used for the detrending. We evaluated the correlation changes of the hyperparameters along the RV time series using the breakpoint technique, further performing a joint photometric and RV analysis using a Markov chain Monte Carlo scheme simultaneously detrending the light curves and the RV time series. We firmly detected the Keplerian signal of TOI-1055 b, deriving a planetary mass of $M_b=20.4_{-2.5}^{+2.6}\,M_{\oplus}$ ($\sim$12%). This value agrees with one of the two estimates in the literature, but it is significantly more precise. Thanks to TESS transit light curve combined with the exquisite CHEOPS photometry, we also derived a planetary radius of $R_b=3.490_{-0.064}^{+0.070}\,R_{\oplus}$ ($\sim$1.9%). Our mass and radius measurements imply a mean density of $\rho_b=2.65_{-0.35}^{+0.37}$ g cm$^{-3}$ ($\sim$14%). We further inferred the planetary structure and found that TOI-1055 b very likely hosts a substantial gas envelope with a mass of $0.41^{+0.34}_{-0.20}$ M$_\oplus$ and a thickness of $1.05^{+0.30}_{-0.29}$ R$_\oplus$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10714-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10714) | **In-orbit performance of LE onboard Insight-HXMT in the first 5 years**  |
|| <mark>Xiaobo Li</mark>, et al. -- incl., <mark>Liming Song</mark>, <mark>Wei Li</mark>, <mark>Shuang-Nan Zhang</mark>, <mark>Haisheng Zhao</mark>, <mark>Maoshun Li</mark>, <mark>Ziliang Zhang</mark>, <mark>Xiaofan Zhao</mark> |
|*Appeared on*| *2023-02-22*|
|*Comments*| *20 pages, 10 figures, submitted*|
|**Abstract**| Purpose: The Low-Energy X-ray telescope (LE) is a main instrument of the Insight-HXMT mission and consists of 96 Swept Charge Devices (SCD) covering the 1-10 keV energy band. The energy gain and resolution are continuously calibrated by analysing Cassiopeia A (Cas A) and blank sky data, while the effective areas are also calibrated with the observations of the Crab Nebula. In this paper, we present the evolution of the in-orbit performances of LE in the first 5 years since launch. Methods: The Insight-HXMT Data Analysis Software package (HXMTDAS) is utilized to extract the spectra of Cas A, blank sky, and Crab Nebula using different Good Time Interval (GTI) selections. We fit a model with a power-law continuum and several Gaussian lines to different ranges of Cas A and blank sky spectra to get peak energies of their lines through xspec. After updating the energy gain calibration in CALibration DataBase (CALDB), we rerun the Cas A data to obtain the energy resolution. An empirical function is used to modify the simulated effective areas so that the background-subtracted spectrum of the Crab Nebula can best match the standard model of the Crab Nebula. Results: The energy gain, resolution, and effective areas are calibrated every month. The corresponding calibration results are duly updated in CALDB, which can be downloaded and used for the analysis of Insight-HXMT data. Simultaneous observations with NuSTAR and NICER can also be used to verify our derived results. Conclusion: LE is a well calibrated X-ray telescope working in 1-10 keV band. The uncertainty of LE gain is less than 20 eV in 2-9 keV band and the uncertainty of LE resolution is less than 15eV. The systematic errors of LE, compared to the model of the Crab Nebula, are lower than 1.5% in 1-10 keV. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10504-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10504) | **Ba-enhanced dwarf and subgiant stars in the LAMOST Galactic surveys**  |
|| <mark>Meng Zhang</mark>, et al. -- incl., <mark>Hua-Wei Zhang</mark>, <mark>Xiao-Wei Liu</mark> |
|*Appeared on*| *2023-02-22*|
|*Comments*| *20 pages, 17 figures. Accepted for publication in ApJ*|
|**Abstract**| Ba-enhanced stars are interesting probes of stellar astrophysics and Galactic formation history. In this work, we investigate the chemistry and kinematics for a large sample of Ba-enhanced ([Ba/Fe]$>$1.0) dwarf and subgiant stars with $5000 < T_{\rm eff }< 6700$\,K from LAMOST. We find that both stellar internal evolution process and external mass exchange due to binary evolution are responsible for the origins of the Ba-enhancement of our sample stars. About one third of them exhibit C and N enhancement and ultraviolet brightness excess, indicating they are products of binary evolution. The remaining Ba-enhanced stars with normal C and N abundances are mostly warm stars with $T_{\rm eff} > 6000$\,K. They are likely consequences of stellar internal elemental transport processes, but they show very different elemental patterns to the hotter Am/Fm stars. Our results reveal a substantially lack of high-[$\alpha$/Fe] Ba-enhanced stars in the [Fe/H]--[$\alpha$/Fe] plane, which we dub as a {\em high-$\alpha$ desert}. We suggest it is due to a lower efficiency for producing Ba-enhanced stars by low-mass AGB progenitors in binary systems. Our results call for detailed modellings for these Ba-enhanced stellar peculiars, in the context of both stellar internal elemental transport and external mass accretion. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10507-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10507) | **Fe and Mg Isotope compositions Indicate a Hybrid Mantle Source for Young  Chang'E 5 Mare Basalts**  |
|| Jiang Y., et al. -- incl., <mark>Liao S. Y.</mark>, <mark>Li P. Y.</mark> |
|*Appeared on*| *2023-02-22*|
|*Comments*| *This paper has already been Accepted by APJL,including 22 pages,9 figures,and 6 tables*|
|**Abstract**| The Chang'E 5 (CE-5) samples represent the youngest mare basalt ever known and provide an access into the late lunar evolution. Recent studies have revealed that CE-5 basalts are the most evolved lunar basalt, yet controversy remains over the nature of their mantle sources. Here we combine Fe and Mg isotope analyses with a comprehensive study of petrology and mineralogy on two CE-5 basalt clasts. These two clasts have a very low Mg# (~29) and show similar Mg isotope compositions with Apollo low-Ti mare basalts as well as intermediate TiO2 and Fe isotope compositions between low-Ti and high-Ti mare basalts. Fractional crystallization or evaporation during impact cannot produce such geochemical signatures which otherwise indicate a hybrid mantle source that incorporates both early- and late-stage lunar magma ocean (LMO) cumulates. Such a hybrid mantle source would be also compatible with the KREEP-like REE pattern of CE-5 basalts. Overall, our new Fe-Mg isotope data highlight the role of late LMO cumulate for the generation of young lunar volcanism. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.10838-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.10838) | **A 2:1 Mean-Motion Resonance Super-Jovian pair revealed by TESS, FEROS,  and HARPS**  |
|| Vladimir Bozhilov, et al. -- incl., <mark>Melissa J. Hobson</mark>, <mark>Thomas Henning</mark>, <mark>Jan Eberhardt</mark>, <mark>Pascal Torres-Miranda</mark>, <mark>Trifon Trifonov</mark> |
|*Appeared on*| *2023-02-22*|
|*Comments*| *Accepted for publication in ApJL. arXiv admin note: text overlap with arXiv:2302.05694*|
|**Abstract**| We report the discovery of a super-Jovian 2:1 mean-motion resonance (MMR) pair around the G-type star TIC 279401253, whose dynamical architecture is a prospective benchmark for planet formation and orbital evolution analysis. The system was discovered thanks to a single transit event recorded by the Transiting Exoplanet Survey Satellite (TESS) mission, which pointed to a Jupiter-sized companion with poorly constrained orbital parameters. We began ground-based precise radial velocity (RV) monitoring with HARPS and FEROS within the Warm gIaNts with tEss (WINE) survey to constrain the transiting body's period, mass, and eccentricity. The RV measurements revealed not one but two massive planets with periods of 76.80$_{-0.06}^{+0.06}$ days and 155.3$_{-0.7}^{+0.7}$ days, respectively. A combined analysis of transit and RV data yields an inner transiting planet with a mass of 6.14$_{-0.42}^{+0.39}$ M$_{\rm Jup}$ and a radius of 1.00$_{-0.04}^{+0.04}$ R$_{\rm Jup}$, and an outer planet with a minimum mass of 8.02$_{-0.18}^{+0.18}$ M$_{\rm Jup}$, indicating a massive giant pair. A detailed dynamical analysis of the system reveals that the planets are locked in a strong first-order, eccentricity-type 2:1 MMR, which makes TIC 279401253 one of the rare examples of truly resonant architectures supporting disk-induced planet migration. The bright host star, $V \approx$ 11.9 mag, the relatively short orbital period ($P_{\rm b}$ = 76.80$_{-0.06}^{+0.06}$ d) and pronounced eccentricity (e =0.448$_{-0.029}^{+0.028}$) make the transiting planet a valuable target for atmospheric investigation with the James Webb Space Telescope (JWST) and ground-based extremely-large telescopes. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 1: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.10528.md
    + _build/html/tmp_2302.10528/./Fig/J00051+457_periodograms.png
    + _build/html/tmp_2302.10528/./Fig/occurrence_rate_comparison_paper.png
    + _build/html/tmp_2302.10528/./Fig/CARMinternalreversed.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\AS}[3]{#1^{+#2}_{-#3}}$
$\newcommand{\Comm}[1]{\textcolor{red}{\textbf{#1}}}$
$\newcommand{\inst}[1]{\unskip^{\instrefs{#1}}}$
$\newcommand{\scsep}{\newcommand{\}{scsep}{,}}$
$\newcommand{\}{scsep}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\AS$}[3]{#1^{+#2}_{-#3}}$
$\newcommand{$\Comm$}[1]{\textcolor{red}{\textbf{#1}}}$
$\newcommand{$\inst$}[1]{\unskip^{$\inst$refs{#1}}}$
$\newcommand{\scsep}{\newcommand{\}{scsep}{,}}$
$\newcommand{\}{scsep}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.10528-b31b1b.svg)](https://arxiv.org/abs/2302.10528)<mark>Appeared on: 2023-02-22</mark> - _Published in A&A (this https URL), 25 pages, 12 figures, Tables 1 and 2 only available online_

</div>
<div id="authors">

I. Ribas, et al. -- incl., <mark><mark>J. I. González Hernández</mark></mark>, <mark><mark>Th. Henning</mark></mark>, <mark><mark>E. Herrero</mark></mark>

</div>
<div id="abstract">

**Abstract:** The CARMENES instrument, installed at the 3.5 m telescope of the Calar Alto Observatory in Almer\'ia, Spain,   was conceived to deliver high-accuracy radial velocity (RV) measurements with long-term stability to search   for temperate rocky planets around a sample of nearby cool stars. Moreover, the broad wavelength coverage   was designed to provide a range of stellar activity indicators to assess the nature of potential RV signals   and to provide valuable spectral information to help characterise the stellar targets.We describe the CARMENES guaranteed time observations (GTO), spanning from 2016 to 2020, during which   19 633 spectra for a sample of 362 targets were collected. We present the CARMENES Data Release 1 (DR1), which   makes public all observations obtained during the GTO of the CARMENES survey.The CARMENES survey target selection was aimed at minimising biases, and about 70\%of all known M   dwarfs within 10 pc and accessible from Calar Alto were included. The data were pipeline-processed, and   high-level data products, including 18 642 precise RVs for 345 targets, were derived.   Time series data of spectroscopic activity indicators were also obtained.We discuss the characteristics of the CARMENES data, the statistical properties of the stellar sample, and the   spectroscopic measurements. We show examples of the use of CARMENES data and provide a contextual view of   the exoplanet population revealed by the survey, including 33 new planets, 17 re-analysed planets, and 26   confirmed planets from transiting candidate follow-up. A subsample of 238 targets was used to derive updated   planet occurrence rates, yielding an overall average of$1.44\pm0.20$planets with 1 M$_\oplus   < M_\mathrm{pl} \sin i < 1000$M$_\oplus$and 1 d$< P_\mathrm{orb} < 1000$d per star, and indicating that   nearly every M dwarf hosts at least one planet. All the DR1 raw data, pipeline-processed data, and high-level   data products are publicly available online.CARMENES data have proven very useful for identifying and measuring planetary companions. They are   also suitable for a variety of additional applications, such as the determination of stellar fundamental   and atmospheric properties, the characterisation of stellar activity, and the study of exoplanet   atmospheres.

</div>

<div id="div_fig1">

<img src="tmp_2302.10528/./Fig/J00051+457_periodograms.png" alt="Fig9" width="100%"/>

**Figure 9. -** Time series data and periodograms for target J00051+457 (GJ 2). {\it Top panel}: NZP-corrected
    RV time series. Outlier measurements (grey symbols; see text) are excluded in the periodogram
    calculation. {\it Bottom panels}(from top to bottom): GLS periodograms of the NZP-corrected RVs (AVC, blue),
    the chromatic index (CRX, black) and CCF bisector span (BIS, red), the differential line width (dLW,
    black) and the CCF FWHM (red), and the chromospheric activity indices corresponding to the H$\alpha$(black) and averaged Ca {\sc ii} IRT lines (red). The dash-dotted blue vertical line in all panels
    marks the position of the most significant peak in the RV periodogram, while the circles in the
    sub-panels highlight the position of the strongest signal in each periodogram, with the period
    given in the legend. The 0.1 \%, 1 \%, and 10 \% FAP levels are shown as horizontal dashed, dash-dotted,
    and dotted  green lines, respectively, and are visible if they fall within the displayed vertical
    range. Vertical axes in panels that show two different datasets are scaled such that the
    FAP levels are identical. The colour code allows the graph to be related with the axis labels and legends. (*fig:sample_periodograms*)

</div>
<div id="div_fig2">

<img src="tmp_2302.10528/./Fig/occurrence_rate_comparison_paper.png" alt="Fig5" width="100%"/>

**Figure 5. -** Comparison of low-mass/small planet occurrence rates from various surveys (adapted from Sab21).
    The parameter range is 1 to 10 M$_\oplus$ in $M_\text{pl} \sin i$ for RV surveys and 1.3 to 3.7 R$_\oplus$ in
    $R_\text{pl}$ for transit surveys. The error bars for transit surveys are Poisson errors based on
    the number of planet detections in the respective bins, and error bars from RV surveys are the 16 \% and
    84 \% levels of the occurrence rate posterior distributions. Results using transiting planets from the
    \emph{Kepler} mission are represented as triangles (\citealt{Morton2014}, \citealt{Dressing2015},
    \citealt{Mulders2015a}, \citealt{Gaidos2016},  \citealt{Hsu2020}), and rates from the HARPS, HADES, and
    CARMENES RV surveys are represented as squares
    \citep[][and this work]{Bonfils2013, Sabotta2021A&A...653A.114S, Pinamonti2022}. The grey
    square shows the occurrence rate from this work with the assumption of a log-uniform distribution in
    $M_\text{pl}\sin i$. (*fig:occurrence_comparison*)

</div>
<div id="div_fig3">

<img src="tmp_2302.10528/./Fig/CARMinternalreversed.png" alt="Fig1" width="100%"/>

**Figure 1. -** Stacked histograms and statistical parameters (median and mode) of the internal precision
    (formal uncertainties) of the 18 642 precise RV measurements in the CARMENES DR1. (*fig:precision*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

213  publications in the last 7 days.
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08736.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


# Debugging papers